# House Price Analysis -- Part One

### *Acknowledgement*
The data set is downloaded as-is from Kaggle [
House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/dataO).<br>

With **79 explanatory variables** describing almost every aspect of residential homes in `Ames, Iowa`.<br> 
The [Ames Housing dataset](http://jse.amstat.org/v19n3/decock.pdf) was compiled by **Dean De Cock** for use in data science education.<br> It's an incredible alternative for data scientists looking for a modernized and expanded version of<br> the often cited Boston Housing dataset[1].

### Dataset Intro

**79 Features** 
1. Quantitative Features 
    - `Continuous variables`: 19
    - `Discrete variables`: 14 
2. Qualitative Features
    - `Nominal variables`: 23
    - `Ordinal variables`: 23
    
**Numeric Target Variable**: `SalePrice`

There are two datasets: `'train.csv'`, and `'test.csv'`. 
1. `'train.csv'` will be used to build and validate the model.
2. `'test.csv'` will be used to predict the final price of each home.

The difference between these two datasets is that `'test.csv'` does NOT have the `Target` variable.

### Purpose of the Project

To use `advanced regression techniques` to build models that predict the final price of each property.<br>
This project will focus on using functions in Python that can easily replicate the analysis process when using the `test.csv`.<br> The first major part of the analysis is on preprocessing and understanding the data set (House Price Analysis-Part One).<br> Then, various models will be built and evaluated on performance.


## Part One: Preprocess the Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Read dataset
def readData(name = 'train.csv'):
    data_df = pd.read_csv(name, keep_default_na = False).drop(columns=['Id'])
    return data_df

In [3]:
data = readData()
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,60,RL,68,11250,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,70,RL,60,9550,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,60,RL,84,14260,Pave,NA,IR1,Lvl,AllPub,FR2,...,0,NA,NA,NA,0,12,2008,WD,Normal,250000


In [4]:
def shapeOfDataSet(df):
    print('The data set has {} rows and {} columns'.format(df.shape[0],\
                                                           df.shape[1]))
shapeOfDataSet(data)

The data set has 1460 rows and 80 columns


### 1. Understand the Unique Values

To ensure there are NO invalid values/typos existing in the columns.<br> 

In [5]:
def uniqueValues(df):
    for col in df.columns:
        if len(df[col].unique()) <= 20:
            print('There are {} unique values in {} column'.format(len(df[col].unique()), col))
            print(df[col].unique(), '\n')

In [6]:
uniqueValues(data) 

There are 15 unique values in MSSubClass column
[ 60  20  70  50 190  45  90 120  30  85  80 160  75 180  40] 

There are 5 unique values in MSZoning column
['RL' 'RM' 'C (all)' 'FV' 'RH'] 

There are 2 unique values in Street column
['Pave' 'Grvl'] 

There are 3 unique values in Alley column
['NA' 'Grvl' 'Pave'] 

There are 4 unique values in LotShape column
['Reg' 'IR1' 'IR2' 'IR3'] 

There are 4 unique values in LandContour column
['Lvl' 'Bnk' 'Low' 'HLS'] 

There are 2 unique values in Utilities column
['AllPub' 'NoSeWa'] 

There are 5 unique values in LotConfig column
['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3'] 

There are 3 unique values in LandSlope column
['Gtl' 'Mod' 'Sev'] 

There are 9 unique values in Condition1 column
['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe'] 

There are 8 unique values in Condition2 column
['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe'] 

There are 5 unique values in BldgType column
['1Fam' '2fmCon' 'Duplex' 'TwnhsE' '

It is notable that feature `MSSubClass` is actually a qualitative variable but encoded as numeric.<br> 
A closer look at this feature, shown below:

`MSSubClass`: Identifies the type of dwelling involved in the sale.

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

It is obvious that the numerical distance between each set of subsequent categories are **NOT** equal.<br>
Therefore, the sensible next step is to understand the data type to further explore and understand the data set.

### 2. Understand the Data Type in the Dataset

In [7]:
# Create a global variable for quantitative predictors in data set
QUANT_VAR_LST = ['LotFrontage','LotArea','OverallQual','OverallCond','YearBuilt',\
                'YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF',\
                'TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea',\
                'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr',\
                'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars',\
                'GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch',\
                'ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']

In [8]:
def originalDataType(df):
    '''
    This function is to uncover the original data type of the data set, 
    and return the following:
    orig_dataType_count - a dataframe showing different dtypes 
                          and the number of features for each type
    dataType_dct - a dictionary with dtype as key and corresponding features as values 
    dataType_key_lst - a list of different dtypes existing in the data set
    dataType_values_lst - a list of corresponding features in each dtype
     
    Parameter:
    df - a pandas DataFrame of the dedicated data set that will be analyzed on
    '''
    if 'SalePrice' in df.columns:
        orig_dataType_count = pd.DataFrame(df.drop(columns=['SalePrice'])\
                                           .dtypes.value_counts(), columns=['Total Amount'])
        dataType_dct = {str(k): list(v) for k, v in df.drop(columns=['SalePrice'])\
                        .groupby(df.dtypes, axis=1)}
    else:
        orig_dataType_count = pd.DataFrame(df.dtypes.value_counts(), columns=['Total Amount'])
        dataType_dct = {str(k): list(v) for k, v in df.groupby(df.dtypes, axis=1)}
    
    dataType_key_lst = list(dataType_dct.keys()) 
    dataType_values_lst = list(dataType_dct.values()) 
    
    return orig_dataType_count, dataType_dct, dataType_key_lst, dataType_values_lst

In [9]:
def referenceFeatureLst(df):  
    '''
    This function will return the following two lists:
    qualitative_var_lst - a list of all correct qualitative features 
    reference_lst - a list of correct quantitative and qualitative features
     
    Parameter:
    df - a pandas DataFrame of the dedicated data set that will be analyzed on
    '''
    if 'SalePrice' in df.columns:
        qualitative_var_lst = [col for col in df.drop(columns=['SalePrice']).columns \
                       if col not in QUANT_VAR_LST]
    else:
        qualitative_var_lst = [col for col in df.columns if col not in QUANT_VAR_LST]
    reference_lst = [QUANT_VAR_LST, qualitative_var_lst]
    return qualitative_var_lst, reference_lst

In [10]:
def incorrectDataType(df):
    '''
    This function will return a list of features that have incorrect dtypes 
     
    Parameter:
    df - a pandas DataFrame of the dedicated data set that will be analyzed on
    '''
    # Uncover original data type for each feature:
    dataType_values_lst = originalDataType(df)[-1]
    reference_lst = referenceFeatureLst(df)[-1]
    
    wrong_feature = [[],[]]
    for i in range(len(dataType_values_lst)):
        # Extract variables that are assigned incorrectly 
        diff_lst = [col for col in dataType_values_lst[i] \
                    if col not in reference_lst[i]]
        wrong_feature[i].extend(diff_lst)
    return wrong_feature

In [11]:
def finalDataTypeCheck(df):
    '''
    This function integrates the above functions to demonstrate the whole process of 
    analyzing the original dtypes of each feature,
    finding inconsistencies by comparing the assigned types with the reference list, 
    and finally identifying the features that have wrong dtypes.
    
    Parameter:
    df - a pandas DataFrame of the dedicated data set that will be analyzed on
    '''
    # Uncover original data type related parameters:
    orig_dataType_count, dataType_dct,\
    dataType_key_lst, dataType_values_lst = originalDataType(df)
    
    qualitative_var_lst = referenceFeatureLst(df)[0]
    
    wrong_feature = incorrectDataType(df)
    
    # Understandoriginal data types in data set
    print('There are {} different data types:\n'.format(len(orig_dataType_count)),\
          orig_dataType_count, '\n')
    
    # Understand correct number of data types
    print('There should be {} qualitative variables'.format(len(qualitative_var_lst)))
    print('There should be {} quantitative variables\n'.format(len(QUANT_VAR_LST)))
    

    for i in range(len(dataType_dct)):    
        if len(wrong_feature) == 0:
            print('All variables have the correct data type!') 
        else:
            print('The following variables are assigned as data type: {}'\
                  .format(dataType_key_lst[i]))
            print(dataType_values_lst[i], '\n')
        
            print('However, there are {} variables should NOT have data type: {}'\
                  .format(len(wrong_feature[i]), dataType_key_lst[i]))
            print(wrong_feature[i], '\n')

In [12]:
finalDataTypeCheck(data)

There are 2 different data types:
         Total Amount
object            46
int64             33 

There should be 44 qualitative variables
There should be 35 quantitative variables

The following variables are assigned as data type: int64
['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'] 

However, there are 1 variables should NOT have data type: int64
['MSSubClass'] 

The following variables are assigned as data type: object
['MSZoning', 'LotFrontage', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'Bld

In [13]:
def correctDataType(df):
    '''
    This function is to correct the wrong dtypes discovered in the above section
 
    Parameter:
    df - a pandas DataFrame of the dedicated data set that will be analyzed on
    '''
    wrong_feature = incorrectDataType(df)
    # Covert features that are supposed to be qualitative to 'object'
    df[wrong_feature[0]] = df[wrong_feature[0]].astype('object')
    # Covert features that are supposed to be quantitative to numeric values
    for i in range(len(wrong_feature[1])):
        df[wrong_feature[1][i]].replace('NA', np.nan, inplace = True)
        df[wrong_feature[1][i]] = pd.to_numeric(df[wrong_feature[1][i]], errors='coerce')
    # Print the dtype of the data set again to ensure accuracy
    orig_dataType_count = originalDataType(df)[0]
    print('After correcting the data type for columns: {},\nthe dtypes of the dataset is:\n{}'\
          .format(wrong_feature[1], orig_dataType_count))

In [14]:
correctDataType(data)

After correcting the data type for columns: ['LotFrontage', 'MasVnrArea', 'GarageYrBlt'],
the dtypes of the dataset is:
         Total Amount
object             44
int64              32
float64             3


### Conclusion:
From the observations, there were some variables incorrectly assgined as `object` that are actually  `numeric features`. <br> This is becuase in Python, some missing values that are represented as strings in a continuous feature can coerce it to <br> read them as object dtypes [2]. For certain columns, having a value of `NA` has meaning (e.g. having no basement).<br> When applying `pd.read_csv()`, the parameter `keep_default_na` was set to False to keep `NA` as a string.<br>A consequence of this, however, is that any legitimate missing values in a numeric column would also contain `NA` as a string,<br> which coereces the entire column to be an `object` dtype. <br>

Therefore, after correcting all features to have the right dtype, those numeric columns should have missing values.<br>The sensible next step should be to calcualte the missing values in the dataset. 

### 3. Understand Missing Values


Create a table that contains **`Total Amount`** and **`Percentage`** of missing values for features that have missing values 

In [15]:
def missValueSummary(df):
    df = pd.DataFrame([df.isnull().sum(), round(df.isnull().mean(),3)])
    df_transpose = df.T  # transpose dataframe, same as df.transpose()
    df_transpose.columns = ['Total Amount', 'Percentage']
    sum_per_df = df_transpose[df_transpose.Percentage > 0]\
                        .sort_values(by='Percentage', ascending = False)
    sum_per_df['Total Amount'] = sum_per_df['Total Amount'].astype('int64')
    if sum_per_df.shape[0] > 0:
        return sum_per_df
    else:
        print('There is no missing value in the data set.')

In [16]:
missValueSummary(data)

,Total Amount,Percentage
LotFrontage,259,0.177
GarageYrBlt,81,0.055
MasVnrArea,8,0.005


## Pre-processing 'test.csv' using Created Functions

In [17]:
data1 = readData('test.csv')
data1.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80,11622,Pave,NA,Reg,Lvl,AllPub,Inside,...,120,0,NA,MnPrv,NA,0,6,2010,WD,Normal
1,20,RL,81,14267,Pave,NA,IR1,Lvl,AllPub,Corner,...,0,0,NA,NA,Gar2,12500,6,2010,WD,Normal
2,60,RL,74,13830,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,0,NA,MnPrv,NA,0,3,2010,WD,Normal
3,60,RL,78,9978,Pave,NA,IR1,Lvl,AllPub,Inside,...,0,0,NA,NA,NA,0,6,2010,WD,Normal
4,120,RL,43,5005,Pave,NA,IR1,HLS,AllPub,Inside,...,144,0,NA,NA,NA,0,1,2010,WD,Normal


In [18]:
uniqueValues(data1) 

There are 16 unique values in MSSubClass column
[ 20  60 120 160  80  30  50  90  85 190  45  70  75 180  40 150] 

There are 6 unique values in MSZoning column
['RH' 'RL' 'RM' 'FV' 'C (all)' 'NA'] 

There are 2 unique values in Street column
['Pave' 'Grvl'] 

There are 3 unique values in Alley column
['NA' 'Pave' 'Grvl'] 

There are 4 unique values in LotShape column
['Reg' 'IR1' 'IR2' 'IR3'] 

There are 4 unique values in LandContour column
['Lvl' 'HLS' 'Bnk' 'Low'] 

There are 2 unique values in Utilities column
['AllPub' 'NA'] 

There are 5 unique values in LotConfig column
['Inside' 'Corner' 'FR2' 'CulDSac' 'FR3'] 

There are 3 unique values in LandSlope column
['Gtl' 'Mod' 'Sev'] 

There are 9 unique values in Condition1 column
['Feedr' 'Norm' 'PosN' 'RRNe' 'Artery' 'RRNn' 'PosA' 'RRAn' 'RRAe'] 

There are 5 unique values in Condition2 column
['Norm' 'Feedr' 'PosA' 'PosN' 'Artery'] 

There are 5 unique values in BldgType column
['1Fam' 'TwnhsE' 'Twnhs' 'Duplex' '2fmCon'] 

There 

In [19]:
finalDataTypeCheck(data1)

There are 2 different data types:
         Total Amount
object            54
int64             25 

There should be 44 qualitative variables
There should be 35 quantitative variables

The following variables are assigned as data type: int64
['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'] 

However, there are 1 variables should NOT have data type: int64
['MSSubClass'] 

The following variables are assigned as data type: object
['MSZoning', 'LotFrontage', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual'

In [20]:
correctDataType(data1)

After correcting the data type for columns: ['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt', 'GarageCars', 'GarageArea'],
the dtypes of the dataset is:
         Total Amount
object             44
int64              24
float64            11


In [21]:
missValueSummary(data1)

,Total Amount,Percentage
LotFrontage,227,0.156
GarageYrBlt,78,0.053
MasVnrArea,15,0.010
BsmtFinSF1,1,0.001
BsmtFinSF2,1,0.001
BsmtUnfSF,1,0.001
TotalBsmtSF,1,0.001
BsmtFullBath,2,0.001
BsmtHalfBath,2,0.001
GarageCars,1,0.001


### References

1. [House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/description)
2. [Handling Categorical Data in Python](https://www.datacamp.com/community/tutorials/categorical-data)
3. [Inconsistent result with replace(np.nan, None, inplace=True)](https://github.com/pandas-dev/pandas/issues/17494)
4. [SettingwithCopyWarning: How to Fix This Warning in Pandas](https://www.dataquest.io/blog/settingwithcopywarning/)